In [2]:
from peft import PeftModel
from transformers import AutoModel, AutoTokenizer, BloomForCausalLM, GenerationConfig
import torch

In [ ]:
BASE_MODEL = "malteos/bloom-6b4-clp-german"

model_bloom= BloomForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="offload",
)
 
tokenizer_bloom= AutoTokenizer.from_pretrained(BASE_MODEL)

In [12]:
model_peft_zubaer = PeftModel.from_pretrained(model_bloom, "abdullahalzubaer/bloom-6b4-clp-german-instruct-lora-peft")

In [4]:
model_peft = PeftModel.from_pretrained(model_bloom, "asprenger/bloom-6b4-clp-german-instruct-lora")

In [13]:
from transformers import AutoTokenizer, GPTJForCausalLM, GenerationConfig

PROMPT = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What are the differences between alpacas and sheep?
### Response:"""

def generate_sth(text):
    # https://colab.research.google.com/drive/1O1JjyGaC300BgSJoUbru6LuWAzRzEqCz?usp=sharing

    inputs = tokenizer_bloom(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0,
        top_p=1,
        repetition_penalty=1.2,
    )

    print("Generating...")
    generation_output = model_peft_zubaer.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=40,
        pad_token_id = 0,
        eos_token_id = 50256
    )

    for s in generation_output.sequences:
        print(tokenizer_bloom.decode(s))
generate_sth(PROMPT)

Generating...
Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
What are the differences between alpacas and sheep?
### Response: Alpakazüchter züchten die Wolle von Tieren, während Schafzüchter nur das Fleisch und den Dung des Tieres nutzen können; Sie haben auch unterschiedliche Körperformen mit einem längeren Hals für Schafe im Vergleich


In [9]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install ipywidgets

In [ ]:
model_peft.push_to_hub("abdullahalzubaer/bloom-6b4-clp-german-instruct-lora-peft", use_auth_token=True)